<a href="https://colab.research.google.com/github/Muskanrathee725/Muskan/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data preprocessing

In [ ]:
# loading the dataset to a panda dataframe


In [14]:
from google.colab import files
import zipfile
import os
import pandas as pd

In [15]:
zip_file = '/content/archive.zip'

In [18]:
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall('/content/fake_news_data')

# Load CSV (replace with actual filename if needed)
df = pd.read_csv('/content/fake_news_data/fake.csv')

# Check data
print(df.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [19]:
fake_news = pd.read_csv('/content/fake_news_data/fake.csv')

In [20]:
true_news = pd.read_csv('/content/fake_news_data/true.csv')

In [24]:
fake_news['label'] = 0  # 0 for fake
true_news['label'] = 1  # 1 for true

data = pd.concat([fake_news, true_news], ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data


In [25]:
stemmer = PorterStemmer()

In [29]:
data.columns

Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')

In [30]:
print(data)

                                                   title  \
0      CARELESS CLINTON AIDE Kept ‘Top Secret’ State ...   
1      Fiore: Bundy Ranch Case Drags on Because ‘FBI ...   
2      China to push for greater cooperation on graft...   
3      Baseball field shooting shakes American symbol...   
4       This Third Grader OBLITERATED Donald Trump In...   
...                                                  ...   
44893  More than half of schools in Boko Haram's regi...   
44894   WATCH: Republican Calls Town Hall Crowd ‘Chil...   
44895   NYC Parks And Rec. BRUTALLY Mocks Trump As Th...   
44896   Senators want Obama to impose sanctions on Congo   
44897  EU sounds alarm, urges U.S. to coordinate on R...   

                                                    text          subject  \
0      How careless are these people? Clinton s right...  Government News   
1      21st Century Wire says Nevada Assemblywoman Mi...      Middle-east   
2      BEIJING (Reuters) - China will push for g

In [32]:
stop_words = set(stopwords.words('english'))

In [33]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z]', ' ', text)  # Remove punctuation and numbers
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

data['clean_text'] = data['text'].apply(preprocess_text)

In [34]:
data['clean_text']

,clean_text
0,careless peopl clinton right hand woman toss d...
1,st centuri wire say nevada assemblywoman miche...
2,beij reuter china push greater intern cooper f...
3,washington reuter gunshot rip congression repu...
4,bunch kid recent ask thought republican front ...
...,...
44893,abuja reuter half school state epicentr nigeri...
44894,republican still push lie afford care act incl...
44895,report earlier statu nake donald trump appear ...
44896,washington reuter u senat call presid barack o...


In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X = tfidf.fit_transform(data['clean_text']).toarray()
y = data['label']


In [36]:
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [38]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.9874164810690423
Confusion Matrix:
 [[4664   65]
 [  48 4203]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4729
           1       0.98      0.99      0.99      4251

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [41]:
def predict_news(text):
    clean = preprocess_text(text)
    vector = tfidf.transform([clean])
    prediction = model.predict(vector)
    return "Fake" if prediction[0] == 0 else "True"

# Example usage:
print(predict_news("Student of Chandigarh University have become billionare"))


Fake
